In [ ]:
import data
import torch
from utils.distmat import *
from utils.evaluation import *
from hitl import *
import numpy as np
import matplotlib.pyplot as plt

## Load Data

In [ ]:
key = data.get_output_keys()[4]
key

In [ ]:
output = data.load_output(key)
qf = torch.Tensor(output["qf"])
gf = torch.Tensor(output["gf"])
q_pids = np.array(output["q_pids"])
g_pids = np.array(output["g_pids"])
q_camids = np.array(output["q_camids"])
g_camids = np.array(output["g_camids"])
distmat = compute_distmat(qf, gf)
q, g = qf.shape[0], gf.shape[0]
q, g

In [ ]:
device = 0
cuda = lambda t: torch.as_tensor(t).cuda(0)

### Baseline Results

In [ ]:
result = evaluate(cuda(distmat), cuda(q_pids), cuda(g_pids), cuda(q_camids), cuda(g_camids), device=device)
result

### Re-ranked Results

In [ ]:
all_distmat = compute_inner_distmat(torch.cat((qf, gf)))
re_distmat = rerank_distmat(all_distmat, qf.shape[0])
re_result = evaluate(re_distmat, q_pids, g_pids, q_camids, g_camids)
re_result

## To CUDA

In [ ]:
qf = cuda(qf)
gf = cuda(gf)
q_pids = cuda(q_pids)
g_pids = cuda(g_pids)
q_camids = cuda(q_camids)
g_camids = cuda(g_camids)

## One-Shot Evaluation Using Modules

### Rocchio

In [ ]:
dm = rocchio.run(qf, gf, q_pids, g_pids, q_camids, g_camids, t=100, device=0)
rocchio_distmat = dm.cpu().numpy()
del dm

### Neighborhood Expansion (Min)

In [ ]:
dm = ne.run(qf, gf, q_pids, g_pids, q_camids, g_camids, t=100, method="min", device=0)
ne_min_distmat = dm.cpu().numpy()
del dm

### Neighborhood Expansion (Mean)

In [ ]:
dm = ne.run(qf, gf, q_pids, g_pids, q_camids, g_camids, t=100, method="mean", device=0)
ne_mean_distmat = dm.cpu().numpy()
del dm

### Neighborhood Expansion (Max)

In [ ]:
dm = ne.run(qf, gf, q_pids, g_pids, q_camids, g_camids, t=100, method="max", device=0)
ne_max_distmat = dm.cpu().numpy()
del dm